In [ ]:
include("init.jl")
using TensorOperations
using TensorDecompositions
using Statistics
using LinearAlgebra
import Plots
import StatsPlots
import LightGraphs # Need to use master to get DAG generator
import GraphPlot
zeroes = zeros

In [ ]:
function g2f(g,r)
    N = LightGraphs.nv(LightGraphs.DiGraph(g))
    base_adj = g |> LightGraphs.adjacency_matrix;
    perms = Int.(rand(N,N,TIMESTEPS) .> LINK_HIATUS_RATE)
    for t in 1:TIMESTEPS
        perms[:,:,t] = perms[:,:,t]  |> Symmetric
    end

    # Want to make a more recognisable temporal pattern
    # Mask a block off and turn it on and off at various times
     perms = zeroes(Int,N,N,TIMESTEPS)
    SPECIAL_NODES = N ÷ 10
    SPECIAL_TIME = 3
    #for i in 1:N, j in 1:N, t in 1:TIMESTEPS
    #    perms[i,j,t] = i <= SPECIAL_NODES && j <= SPECIAL_NODES && t <= SPECIAL_TIME ? 0 : 1
    #end
    # should probably find a nicer way of doing this...

    ADJ = zeroes(N,N,TIMESTEPS) # making this takes _AGES_
    #for i in 1:N, j in 1:N, t in 1:TIMESTEPS
    #    ADJ[i,j,t] = base_adj[i,j] & perms[i,j,t]
    #end

    for t in 1:TIMESTEPS
        ADJ[:,:,t] = base_adj
    end

    # I expected
    # @tensor ADJ[i,j,t] = base_adj[i,j] & perms[i,j,t]
    # to work, but it doesn't.

    # should probably try to make more distinct temporal communities that agree with SBM to make this a fairer test

    ADJ |> size

    nncp(ADJ, r; # R is the number of components ≈ number of communities
        # I think Ciro spoke about optimising this number somewhere
        # papers refer to it - core consistency
        compute_error=true,
        maxiter=1000,
        verbose=false
    )
end

function g2coms(g,fac)
    F = g2f(g,3)
    Plots.heatmap(
        F.factors[fac]; # Hmm, so Colin was right, it's because of the power law
        xlabel="Community",
        ylabel="Node membership strength",
    )
end

In [ ]:
N = 100 # ignored if using SBM
TIMESTEPS = 10
DENSITY = 0.05 # unused atm
LINK_HIATUS_RATE = 0.1


In [ ]:
g = LightGraphs.barabasi_albert(N,1)

base_adj = g |> LightGraphs.adjacency_matrix |> UpperTriangular;
g = LightGraphs.DiGraph(base_adj)
g2coms(g,2)

In [ ]:
g2coms(g,1)

In [ ]:
g = LightGraphs.barabasi_albert(N,1)
g2coms(g,1)

In [ ]:
g2coms(g,2)

In [ ]:
g = LightGraphs.erdos_renyi(N,0.05)
g2coms(g,2)

In [ ]:
g2coms(g,1)

In [ ]:
g = LightGraphs.random_orientation_dag(g)

g2coms(g,1)

In [ ]:
g2coms(g,2)

In [ ]:
g = LightGraphs.stochastic_block_model(
    [
        3   2/N   2/N; # c[a,b] = mean number of neighbours between nodes in block a and block b (only pay attention to upper tri)
        0   3   2/N;
        0   0   3;
    ],
    [50,50,50] # number of nodes in each block
)
g2coms(g,1)

In [ ]:
g2coms(g,2)

In [ ]:
g = LightGraphs.random_orientation_dag(g)
g2coms(g,1)

In [ ]:
g2coms(g,2)

In [ ]:
base_adj = g |> LightGraphs.adjacency_matrix |> UpperTriangular;
g = LightGraphs.DiGraph(base_adj)
g2coms(g,1)

In [ ]:
g2coms(g,2)

In [ ]:
# So, in conclusion: we get mostly empty first factor because of power law + upper-triangular arrangement,
# Not really anything to do with DAG

In [ ]:
g = LightGraphs.barabasi_albert(N,1)

base_adj = g |> LightGraphs.adjacency_matrix |> UpperTriangular;
g = LightGraphs.DiGraph(base_adj)
F = g2f(g,3)

In [ ]:
@show mean(F.factors[1])
@show Statistics.std(F.factors[1])
@show Statistics.median(F.factors[1])

In [ ]:
@show mean(F.factors[2])
@show Statistics.std(F.factors[2])
@show Statistics.median(F.factors[2])

In [ ]:
Plots.boxplot(normalize(F.factors[1][F.factors[1] .< mean(F.factors[1])])) # Power law all the way down